In [22]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    # other params...
)


In [23]:
a = llm.invoke("who is the prime minister of india")
print(a)

content='As of my cut-off knowledge in December 2023, the Prime Minister of India is Narendra Modi. He has been serving as the Prime Minister of India since May 26, 2014. However, please note that my information may not be up-to-date, and I recommend verifying the current Prime Minister of India through a reliable news source for the most recent information.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 42, 'total_tokens': 118, 'completion_time': 0.101333333, 'prompt_time': 0.003760913, 'queue_time': 0.017284113, 'total_time': 0.105094246}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None} id='run-c21dfe3e-14ce-426e-9465-7acc3927bc89-0' usage_metadata={'input_tokens': 42, 'output_tokens': 76, 'total_tokens': 118}


In [57]:
from langchain_community.document_loaders import YoutubeLoader

youtube_url = "https://youtu.be/zIwLWfaAg-8?si=qsN6eA17rsIqJl8o"
loader = YoutubeLoader.from_youtube_url(youtube_url=youtube_url, add_video_info=False)
docs = loader.load()

In [61]:
print(docs)

[Document(metadata={'source': 'zIwLWfaAg-8'}, page_content='Chris Anderson:\nElon, hey, welcome back to TED. It\'s great to have you here. Elon Musk: Thanks for having me. CA: So, in the next half hour or so, we\'re going to spend some time exploring your vision for what\nan exciting future might look like, which I guess makes\nthe first question a little ironic: Why are you boring? EM: Yeah. I ask myself that frequently. We\'re trying to dig a hole under LA, and this is to create the beginning of what will hopefully\nbe a 3D network of tunnels to alleviate congestion. So right now, one of the most\nsoul-destroying things is traffic. It affects people\nin every part of the world. It takes away so much of your life. It\'s horrible. It\'s particularly horrible in LA. (Laughter) CA: I think you\'ve brought with you the first visualization\nthat\'s been shown of this. Can I show this? EM: Yeah, absolutely.\nSo this is the first time -- Just to show what we\'re talking about. So a couple of

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)
docs = text_splitter.split_documents(docs)

In [60]:
docs

[Document(metadata={'source': 'zIwLWfaAg-8'}, page_content='Chris Anderson:\nElon, hey, welcome back to TED. It\'s great to have you here. Elon Musk: Thanks for having me. CA: So, in the next half hour or so, we\'re going to spend some time exploring your vision for what\nan exciting future might look like, which I guess makes\nthe first question a little ironic: Why are you boring? EM: Yeah. I ask myself that frequently. We\'re trying to dig a hole under LA, and this is to create the beginning of what will hopefully\nbe a 3D network of tunnels to alleviate congestion. So right now, one of the most\nsoul-destroying things is traffic. It affects people\nin every part of the world. It takes away so much of your life. It\'s horrible. It\'s particularly horrible in LA. (Laughter) CA: I think you\'ve brought with you the first visualization\nthat\'s been shown of this. Can I show this? EM: Yeah, absolutely.\nSo this is the first time -- Just to show what we\'re talking about. So a couple of

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Tuple
from langchain_google_genai import ChatGoogleGenerativeAI

class QuizQuestion(BaseModel):
    """
    Structured model for quiz questions and answers.
    """
    question: str = Field(description="The quiz question")
    option1: str = Field(description="First option")
    option2: str = Field(description="Second option")
    option3: str = Field(description="third option")
    option4: str = Field(description="Fourth option")
    correct_answer: str = Field(description="The correct answer to the question")

class QuizSet(BaseModel):
    """
    Collection of quiz questions.
    """
    questions: List[QuizQuestion] = Field(description="List of quiz questions with their answers")

def generate_quiz_questions(docs: str) -> List[List[str]]:
    """
    Generate quiz questions from provided text documentation.
    Returns a list of questions, each with one correct and two incorrect answers.
    """
    output_parser = JsonOutputParser(pydantic_object=QuizSet)
    
    prompt = PromptTemplate(
        template="""
        You are a quiz generator tasked with creating questions based on the following text.
        Generate 5 distinct questions, each with one correct answer and two plausible but incorrect answers.
        
        Text to analyze:
        {input_text}
        
        {format_instructions}
        
        Generate questions that:
        - Test understanding of key concepts
        - Have clear, unambiguous correct answers
        - Include plausible but clearly incorrect alternative answers
        - Cover different aspects of the provided text
        
        Return the questions in a structured format where each question is accompanied by its correct answer
        and two incorrect alternatives.
        """,
        input_variables=["input_text"],
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
    
    chain = prompt | llm | output_parser
    
    try:
        # Generate questions using the LLM
        result = chain.invoke({"input_text": docs})
        return result
    
    except Exception as e:
        print(f"Error in quiz generation: {e}")
        return []


In [39]:
a = generate_quiz_questions(docs)

In [43]:
from access_json import save_json,load_json
import os
path = os.path.join("result.json")
save_json(a,path)

Data successfully saved to result.json


In [54]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict
import os


class QuizQuestion(BaseModel):
    """
    Structured model for quiz questions and answers.
    """
    question: str = Field(description="The quiz question")
    options: List[str] = Field(description="List of answer options, including the correct one and incorrect alternatives")
    correct_answer: str = Field(description="The correct answer to the question")


class QuizSet(BaseModel):
    """
    Collection of quiz questions.
    """
    questions: List[QuizQuestion] = Field(description="List of quiz questions with their options and correct answers")


def generate_quiz_questions(docs: str) -> List[Dict[str, str]]:
    """
    Generate quiz questions from provided text documentation.
    Returns a list of dictionaries, each containing a question, options, and correct answer.
    """
    output_parser = JsonOutputParser(pydantic_object=QuizSet)

    prompt = PromptTemplate(
        template="""
        You are a quiz generator tasked with creating questions based on the following text.
        Generate 5 distinct questions, each with one correct answer and two plausible but incorrect answers.

        Text to analyze:
        {input_text}

        {format_instructions}

        Generate questions that:
        - Test understanding of key concepts
        - Have clear, unambiguous correct answers
        - Include plausible but clearly incorrect alternative answers
        - Cover different aspects of the provided text

        Return the questions in a structured JSON format where:
        - Each question has a "question" field
        - An "options" field containing a list of answers (1 correct and 2 incorrect answers)
        - A "correct_answer" field that explicitly states the correct answer
        """,
        input_variables=["input_text"],
        partial_variables={"format_instructions": output_parser.get_format_instructions()}
    )

    # Replace with your own LLM configuration
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

    chain = prompt | llm | output_parser

    try:
    #     # Generate questions using the LLM
        result = chain.invoke({"input_text": docs})
        
    #     # Convert the structured output to the desired format
    #     quiz_json = [
    #         {
    #             "question": q.question,
    #             "options": q.options,
    #             "correct_answer": q.correct_answer
    #         }
    #         for q in result.questions
    #     ]
        
        return result

    except Exception as e:
        print(f"Error in quiz generation: {e}")
        return []


# Example usage:
# docs = "Your input text here..."
# quiz_questions = generate_quiz_questions(docs)
# print(quiz_questions)


In [56]:
otpt = generate_quiz_questions(docs)
from access_json import save_json,load_json
import os
path = os.path.join("result.json")
save_json(otpt,path)

Data successfully saved to result.json


In [47]:
a = load_json("result.json")
a["questions"]

Data successfully loaded from result.json


[{'question': "What is the primary goal of Elon Musk's Boring Company?",
  'option1': 'To develop faster-than-sound transportation',
  'option2': 'To alleviate urban traffic congestion through a 3D network of tunnels',
  'option3': 'To create a global network of underground hyperloops',
  'correct_answer': 'To alleviate urban traffic congestion through a 3D network of tunnels'},
 {'question': 'How does Elon Musk propose to significantly reduce the cost of tunnel construction?',
  'option1': 'By using robots to perform all aspects of tunnel construction',
  'option2': 'By reducing tunnel diameter and implementing continuous tunneling and reinforcing',
  'option3': 'By developing new, stronger materials for tunnel walls',
  'correct_answer': 'By reducing tunnel diameter and implementing continuous tunneling and reinforcing'},
 {'question': "What is Elon Musk's main argument against flying cars as a solution to traffic congestion?",
  'option1': 'Flying cars are too expensive to produce o